In [16]:



def get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name):
    from Bio import SeqIO
   
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")

    if "Period_gene_genomic_sequence_individual_exon" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon")

    list_of_files_inside_indiv_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon")
    if "temp" not in list_of_files_inside_indiv_exon_folder:
         os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp")

    os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}"')
    os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp/temp_genome.fa"')
    genome = SeqIO.parse(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp/temp_genome.fa", "fasta")
    # genome = SeqIO.parse("/mnt/c/Users/sauba/Downloads/Compressed/ncbi_dataset_7/ncbi_dataset/data/GCA_917051295.2/GCA_917051295.2_ilEreLige1.2_genomic.fna", 'fasta')
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    return (gene_sequence)

def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)   

def get_sequence(blast_output_location,genome_location, species, genome_file,annotated_genome_location,annotated_species_name):
    list_of_files_in_species_folder = os.listdir(f"{blast_output_location}/{species}")
    coordinate_file_name = ''
    for file_names in list_of_files_in_species_folder:
        if file_names.endswith("_coordinates.csv"):
            coordinate_file_name = file_names

    with io.open(f"{blast_output_location}/{species}/{coordinate_file_name}", 'r') as temp_file_open:
        coordinate_file_lines = temp_file_open.readlines()

    upstream_exon_line =coordinate_file_lines[1].split(",")
    if upstream_exon_line[5] == "Y":
        print("First Exon Error!!")
        assert False

    downstream_exon_line = coordinate_file_lines[-1].split(",")
    if downstream_exon_line[5] == "Y":
        print("Last Exon Error!!")
        assert False

    
    complement,scaffold = upstream_exon_line[4], upstream_exon_line[1]

    if complement == "0":
        gene_start = min(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
        
        gene_end = max(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
        
    
    if complement == "1":
        gene_start = min(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
        gene_end = max(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
    
    padding_error = ''
    
    if gene_start < pad_seq:
        print(f"Error{gene_start}\n Changed Padding")
        print(species)
        assert False
    gene_start -= pad_seq
    gene_end +=  pad_seq
    
    gene_sequence = get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name)
    print("BB")
    return(gene_sequence,scaffold,gene_start,gene_end,padding_error )

def get_annotated_genome_name(annotated_genome_location, species):
    list_of_annotated_genomes = os.listdir(annotated_genome_location)
  
    # print(annotated_species)
    for annotated_species in list_of_annotated_genomes:
        if annotated_species.endswith(species):
            return(annotated_species)
             
    if annotated_species_name == '':
        print(f"Error with annotated species name")
        assert False



In [17]:
def make_species_folder(output_location, species):
    list_of_folders = os.listdir(f"{output_location}/5.For Email_padded/")
    if species not in list_of_folders:
#         subprocess.run(f'rm -r "{output_location}/5.For Email_padded/{species}"', shell = True, stderr = subprocess.DEVNULL)
#         import time

#         time.sleep(5)
        os.mkdir(f"{output_location}/5.For Email_padded/{species}")


In [ ]:
# family_group_list = ["3.Satyrine","4.Pierinae","5.Coliadinae","6.Heliconiinae_Danainae_Nymphalinae",
family_group_list =  ["6.Heliconiinae_Danainae_Nymphalinae"]
for family_group in family_group_list:

    # family_group = "9.Papilionidae"
    pad_seq = 10000


    import os
    import io
    import subprocess
    genome_location =  "/mnt/f/Genomes_2023-12-26"
    output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}"

    list_of_folders = os.listdir(f"{output_location}")
    if "5.For Email_padded" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/5.For Email_padded"', shell = True)

    os.mkdir(f"{output_location}/5.For Email_padded")

    blast_output_location = f"{output_location}/1.Blast_result"

    list_of_species = os.listdir(f"{output_location}/2.Final_output")
    if "desktop.ini" in list_of_species:
        list_of_species.remove('desktop.ini')

    error_species = ''
    # list_of_species.remove("Colias_nastes")
    # list_of_species = ["Colias_nastes"]
    for species in list_of_species:
        print(species)
        if species == "Papilio_glaucus":
            pad_seq = 1000
        else:
            pad_seq = 10000
        make_species_folder(output_location, species)
        annotated_genome_location = f"{output_location}/1.Blast_result"

        genome_file = get_genome_file(genome_location,species)
        annotated_species_name = get_annotated_genome_name(annotated_genome_location, species)

        gene_sequence,scaffold, gene_start, gene_end, padding_error = get_sequence(blast_output_location,genome_location, species, genome_file,
                                                                   annotated_genome_location,annotated_species_name)
        error_species += f"{padding_error}\n"
    #     print(gene_sequence,scaffold, gene_start, gene_end)
        with open(f"{output_location}/5.For Email_padded/{species}/Genome_fragment.fa",'w') as out_file:
            output = f">{species}_{scaffold}_{gene_start}_{gene_end}\n{gene_sequence}"
            out_file.write(output)

    with open(f"{output_location}/5.For Email_padded/error_species.txt",'w') as error_out_file:
        error_out_file.write(error_species)


    #     local_genomic_fragment_location = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/"
    #     cd_command = f'cd "{local_genomic_fragment_location}"\nmakeblastdb -in gene_sequence_all.fa -dbtype nucl\n'
    #         # # os.system(f'{cd_command}')
    #     subprocess.run(f'{cd_command}', shell = True, stderr = subprocess.DEVNULL)
    #         # # print(mkdb_command)
    #         # # os.system(f'{mkdb_command}')
    #     # print(gene_sequence)


    import os
    from Bio import SeqIO

    # output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/{family_group}"

    # list_of_species = os.listdir(f"{output_location}/2.Final_output")
    # if "desktop.ini" in list_of_species:
    #     list_of_species.remove('desktop.ini')
    # print(list_of_species)
    # list_of_species = ["Anthocharis_cardamines"]
    for species in list_of_species:
        print(species)
        if species == "Papilio_glaucus":
            pad_seq = 1000
        else:
            pad_seq = 10000
        
    #     print(gene_start)
        output = ''
        genome_fragment_file = SeqIO.parse(f"{output_location}/5.For Email_padded/{species}/Genome_fragment.fa", "fasta")
        for sequences in genome_fragment_file:
            genome_name = sequences.id
            fragment_length = len(sequences.seq)
            print(fragment_length)
        list_of_variant_files = os.listdir(f"{output_location}/4.Bed File/{species}/")
        if 'desktop.ini' in list_of_variant_files:
            list_of_variant_files.remove('desktop.ini')
    #     start = 0

        for variant in list_of_variant_files:    
            with open(f"{output_location}/4.Bed File/{species}/{variant}", 'r') as var_file:
                var_list = var_file.readlines()

            if int((var_list[0].split("\t")[1])) <int((var_list[1].split("\t")[1])):
                complement = 0
                gene_start = int((var_list[0].split("\t")[1]))
            else:
                complement = 1
                gene_start = int((var_list[-1].split("\t")[1])) 
            output = ''
    #         print(var_list)
            for gene_line in  var_list:

                gene_line_split = gene_line.split("\t")
    #             print(gene_line, var_list)
                gene_line_split[0] = genome_name
                if complement == 0:
                    gene_line_split[1] = str(int(gene_line_split[1]) - gene_start + pad_seq)
                    gene_line_split[2] = str(int(gene_line_split[2]) - gene_start + pad_seq)
                if complement == 1:
                    old_gene_line_split_1 = gene_line_split[1]
                    gene_line_split[1] = str(fragment_length - (int(gene_line_split[2]) - gene_start + pad_seq))
                    gene_line_split[2] = str(fragment_length - (int(old_gene_line_split_1) - gene_start + pad_seq))
                output += "\t".join(gene_line_split)

    #         print(output)
            with open(f"{output_location}/5.For Email_padded/{species}/{variant}", 'w') as out_file:
                out_file.write(output)
    #         print(complement, gene_start)
    #         assert False


Mechanitis_mazaeus


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


BB
Mechanitis_messenoides


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


BB
Melinaea_marsaeus_rileyi


...

BB
Danaus_plexippus


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


BB
Dircenna_loreta


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


BB
Mechanitis_mazaeus
29030
Mechanitis_messenoides
28934
Melinaea_marsaeus_rileyi
29782
Melinaea_menophilus
29092
Napeogenes_inachia
26815
Napeogenes_sylphis
26772
Argynnis_bischoffii_washingtonia
33479
Boloria_euphrosyne
31874
Boloria_selene
32467
Brenthis_daphne
34041
Brenthis_hecate
39946
Brenthis_ino
39104
Dryadula_phaetusa
31729
Dryas_iulia_moderata
34163
Eueides_isabella
34046
Fabriciana_adippe
42872
Heliconius_charithonia
30595
Heliconius_nattereri
29574
Heliconius_sara
32089
Philaethria_dido
33717
Limenitis_camilla
32330
Euphydryas_editha
36999
Hypolimnas_misippus
32428
Melitaea_cinxia
35085
Mellicta_athalia
37116
Nymphalis_c-album
30381
Nymphalis_io
31721
Nymphalis_polychloros
30386
Nymphalis_urticae
29989
Vanessa_atalanta
28460
Vanessa_cardui
29951
Vanessa_tameamea
28464
Hestina_assimilis
32723
Danaus_plexippus
28193
Dircenna_loreta
26731
